In [ ]:
from clustering import *
import datetime
import os
import warnings
from optimization import v2gOptimization

warnings.simplefilter(action='ignore', category=FutureWarning)

## Define path to Data folder

In [ ]:
path = os.getcwd() + '/Data/'

## Create baseline charging data files - only needs to be run once

In [ ]:
# load the initial data
filename = path + 'audi_etron_ca_SAMPLE_PROCESSED_withAccess_withSpeeds.csv'
vehicle_df_all = load_data(filename)

In [ ]:
# create baseline data for a specific date range
min_date = datetime.date(2020, 1, 1)
max_date = datetime.date(2020, 1, 31)
data_subset = create_baseline_data(vehicle_df_all, min_date, max_date)

## Create feature array

In [ ]:
#create weekly feature array
min_date = datetime.date(2019, 10, 1)
max_date = datetime.date(2020, 2, 29)
feature_type = 'access_daily_weekdays'
feature_array, vinids = create_feature_array(min_date, max_date, feature_type, path)

In [ ]:
#save vinids and feature array
print('Feature array shape is: ' + feature_array.shape)
pd.DataFrame(vinids).to_csv(path + 'vinids.csv', index=True)
pd.DataFrame(feature_array).to_csv(path + 'feature_array_daily_access.csv', index=False)

## Load feature arrays and run clustering

In [ ]:
#run adaptive clustering
feat_array = pd.read_csv(path + 'feature_array_daily_access.csv').values
run_adaptive_clustering(feat_array)

## Check if MOSEK license is present (Windows)

In [ ]:
myusername = 'myusername'  # Replace with your actual username
if os.path.exists(f'C:\\Users\\{myusername}\\mosek\\mosek.lic'):
    print('License file is present and optimization can proceed.')
else:
    print('License file is missing. Please ensure that the license file is in the correct directory.')


## Check if MOSEK license is present (Mac)

In [ ]:
myusername = 'myusername'  # Replace with your actual username
if os.path.exists('/Users/'+myusername+'/mosek/mosek.lic'):
    print('License file is present and optimization can proceed.')
else:
    print('License file is missing. Please ensure that the license file is in the correct directory.') 

## Run optimization

In [ ]:
#Set initial parameters for optimization
circuit = 1
battery_cost = 0
day_ahead = False
num_vins = 10

#get parameters file
params_filename = 'Parameters/params_pge_'+str(battery_cost) + '_batt_' + str(circuit) +'_cir.json'
batt_params_filename = 'Parameters/params_battery_model.json'
v2g_opt = v2gOptimization(params_filename, batt_params_filename)

#run optimization
vinids = np.arange(1,num_vins)
if day_ahead:
    for vin in vinids:
        print('Running day-ahead optimization for vin ' + str(vin))
        v2g_opt.call_optimization_day_ahead(vin)
else:
    for vin in vinids:
        print('Running weekly perfect foresight optimization for vin ' + str(vin))
        v2g_opt.call_optimization(vin)